In [1]:
import pandas

In [2]:
#path = "~/Projects/AL_HTML/examples/FracPreBake/FractionArithmetic/converted_control.txt"
#path = "/home/danny/Downloads/ds649_tx_n_4299_2018_0418_004541.txt"
#df = pandas.read_csv(path, sep='\t', lineterminator='\n', skip_blank_lines=True).replace({r'\r': ''}, regex=True)
#df = df.rename(index=str, columns={a:a.rstrip() for a in df.keys()})
##print(df)
#print("-----------------")
#path = "~/Projects/AL_HTML/examples/FracPreBake/FractionArithmetic/converted_human.txt"
#df_human = pandas.read_csv(path, sep='\t', lineterminator='\n', skip_blank_lines=True).replace({r'\r': ''}, regex=True)
#df_human = df_human.rename(index=str, columns={a:a.rstrip() for a in df.keys()})
#print(df_human)

In [3]:
#student_tables = []
#for x in df['Anon Student Id'].unique():
#    student_tables.append(df[df['Anon Student Id'] == x])
#print([x.shape for x in student_tables])
#student_tables_human = []
#for x in df_human['Anon Student Id'].unique():
#    student_tables_human.append(df_human[df_human['Anon Student Id'] == x])
#print([x.shape for x in student_tables_human])

In [4]:

#print(t.keys())
import copy
from pprint import pprint
def generate_step_slices(x, KC='KC (Field)'):
    outcomes = x['Outcome']
    selections = x[KC]
    d = {}
    #last_correct = 0
    for row in range(x.shape[0]):
        selection = str(selections[row])
        if(selection == ""):
            continue
        l = d.get(selection,[])   
        d[selection] = l + [x.iloc[row]]
        
        if(str(outcomes[row]).strip().upper() == "CORRECT"):
            
            yield pandas.DataFrame(d[str(selections[row])],columns=x.keys())
            del d[str(selections[row])]
            #yield x[last_correct:row+1]
            #last_correct = row+1
        
            

            
WRONG_SELECTION ="Instead of the step you are working on, please work on the highlighted step."
NOT_DONE = "I'm sorry, but you are not done yet. Please continue working."
def generate_step_dicts(student_table, KC='KC (Field)',only_first_error=True):
    opportunity_counts = {}
    for step_slice in generate_step_slices(student_table):
        
        #print(step_slice[["Selection",'Input','Outcome']])
        #print("-------------------------------")
        correct_selection = step_slice['Selection'].iloc[-1]
        correct_input = step_slice['Input'].iloc[-1]
        step_name = step_slice[KC].iloc[-1]
        if(not isinstance(step_name,str) or step_name == ""):
            continue
        
        d = {}
        for _,table_row in step_slice.iterrows():
            #print("\n")
            #print(table_row)
            if(str(table_row['Outcome']).strip().upper() == "INCORRECT"):
                #print(table_row['Selection'], table_row['Action'], table_row['Input'])
                
                #if(table_row['Selection'] != correct_selection):
                if(table_row["Feedback Text"] == WRONG_SELECTION or table_row["Feedback Text"] == NOT_DONE):
                    d["selection_error"] = True
                elif(table_row['Input'] != correct_input):
                    d["input_error"] = True
            elif(str(table_row['Outcome']).strip().upper() == "HINT"):
                d["hint"] = True
            if(only_first_error):
                break
        d['KC'] = step_name 
        
        opportunity_counts[step_name] = opportunity_counts.get(step_name,0) + 1
        d['Opportunity'] = opportunity_counts[step_name]
        
        yield d
        #print("CORRECT", correct_selection,step_slice['Action'].iloc[-1], correct_input)
        #print(d)
        #print("\n")
        
def increment_at_index(l,i,amount=0):
    if(i < len(l)):
        l[i] += amount
    elif(i == len(l)):
        l.extend([amount])
    else:
        raise ValueError("Nope", len(l), i)
        
                
def get_learning_curves(student_tables, KC='KC (Field)',only_first_error=True):
    kc_dict = {}
    kc_template = {"Counts": [], "Errors": [], "Selection Errors": [], "Input Errors": [], "Hint Errors" : []}
    
    
    for st in student_tables:
        for sd in generate_step_dicts(st, KC=KC,only_first_error=only_first_error):
            kc_name = sd.get('KC',None)
            kc_data = kc_dict.get(kc_name, copy.deepcopy(kc_template))
            
            
            
            increment_at_index(kc_data["Counts"], sd["Opportunity"] -1, 1)
            
            se,ie, h = sd.get('selection_error',False), sd.get('input_error',False),sd.get('hint',False)
            op = sd["Opportunity"]
            indx = op-1
            #print(indx, sd)
            #print(kc_name, indx, kc_data)
            
            
            increment_at_index(kc_data["Errors"], indx, 1 if se or ie or h else 0)
            
            increment_at_index(kc_data["Selection Errors"], indx, 1 if se else 0)
            
            increment_at_index(kc_data["Input Errors"], indx, 1 if ie else 0)
            
            increment_at_index(kc_data["Hint Errors"], indx, 1 if h else 0)
            
            kc_dict[kc_name] = kc_data
                    
    return kc_dict

def gen_student_tables(df):
    student_tables = []
    for x in df['Anon Student Id'].unique():
        student_tables.append(df[df['Anon Student Id'] == x])
    return student_tables

def transaction_file_to_df(path):
    df = pandas.read_csv(path, sep='\t', lineterminator='\n', skip_blank_lines=True).replace({r'\r': ''}, regex=True)
    df = df.rename(index=str, columns={a:a.rstrip() for a in df.keys()})
    return df
            
def curves_from_transaction_file(df, KC='KC (Field)',only_first_error=True):
    if(not isinstance(df, pandas.DataFrame)):
        if(isinstance(df, str)):
            #path = "/home/danny/Downloads/ds649_tx_n_4299_2018_0418_004541.txt"
            df = transaction_file_to_df(df)
        else:
            raise ValueError("Transaction file must be path or pandas.DataFrame, got %s" % type(df))
        
        
    student_tables = gen_student_tables(df)
    learning_curves_by_kc = get_learning_curves(student_tables, KC=KC,only_first_error=only_first_error)
    return learning_curves_by_kc
    

                
        #print(step_slice['Selection'])

        #print(correct_selection, correct_input)

        


In [ ]:
#t = student_tables[0]
#learning_curves_by_kc = get_learning_curves(student_tables)

path = "human_converted_v2.txt"
human_curves = curves_from_transaction_file(path)
del human_curves["AS check_convert"]
del human_curves["M check_convert"]

path = "converted_control_id_fix_time_fix.txt"
control_curves = curves_from_transaction_file(path)
#pprint(learning_curves_by_kc,width=200)
#print("--------")


path = "converted_pretest_time_fix.txt"
pretest_curves = curves_from_transaction_file(path)

path = "substep_converted_v3_id_fix_time_fix.txt"
substep_curves = curves_from_transaction_file(path)

path = "iso_converted_v4_id_fix_time_fix.txt"
iso_curves = curves_from_transaction_file(path)



In [ ]:
path = "refactorControl_converted_v2_id_fix_time_fix.txt"
refactor_curves = curves_from_transaction_file(path)
#pprint(learning_curves_by_kc_human,width=200)

In [ ]:
path = "light_refactor_control_converted_id_fix_time_fix.txt"
light_refactor_curves = curves_from_transaction_file(path)

In [ ]:
path = "reintro_bug_control_converted.txt"
reintro_bug_curves = curves_from_transaction_file(path)

In [ ]:
path = "final_test_converted.txt"
final_curves = curves_from_transaction_file(path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors

def graph_split(name, kc,count_thresh=10,ax=None,show=True,ylim=[-0.01,1.025]):
    colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
    
    counts = np.array(kc['Counts'], dtype=np.float)
    error = np.array(kc['Errors'], dtype=np.float)/counts
    s_error = np.array(kc['Selection Errors'], dtype=np.float)/counts
    i_error = np.array(kc['Input Errors'], dtype=np.float)/counts
    h_error = np.array(kc['Hint Errors'], dtype=np.float)/counts
    
    print(counts)
    
    fig = plt.figure()
    
    if(not ax): ax = plt.subplot(111) 
    
    
    x = np.arange(len(counts))+1
    
    ax.plot(x, error  , marker="o" ,label='total error', color=colors['firebrick'])
    ax.plot(x, s_error, marker="d" ,label='selection error', color=colors['gold'])
    ax.plot(x, i_error, marker="s" , label='input error', color=colors['seagreen'])
    ax.plot(x, h_error, marker="1" ,label='hint error', color=colors['dodgerblue'])
    
    ax.set_ylim([-0.01,.4])
    #ax.set_ylim([-0.01,1.025])
    
    ax.legend()
    plt.minorticks_on()
    ax.grid(alpha=.5,which='major')
    #ax.grid(alpha=.2,which='minor')
    
    plt.title(name)
    plt.ylabel('Average Error')
    plt.xlabel('opportunity')
    if(show): plt.show()
    
    return ax
    

def graph_kcs(name, kcs,count_thresh=10):
    fig = plt.figure()
    ax = plt.subplot(111)    
    
    colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)
    l_colors = ['firebrick', 'gold' ,'seagreen', 'dodgerblue', 'darkorchid']
    l_markers = ['o', 'd', 's', '|', 'P']
    
    for i, (kc_name, kc) in enumerate(kcs.items()):
        kc = apply_threshold(kc, count_thresh)
        
        counts = np.array(kc['Counts'], dtype=np.float)
        error = np.array(kc['Errors'], dtype=np.float)/counts
        x = np.arange(len(counts))+1
        
        ax.plot(x, error  , marker=l_markers[i] ,label=kc_name, color=colors[l_colors[i]])
              
    ax.set_ylim([-0.01,1.025])
    #ax.set_ylim([-0.01,22])
    
    ax.legend()
    plt.minorticks_on()
    ax.grid(alpha=.5,which='major')
    ax.grid(alpha=.2,which='minor')
    
    
    if(name != None): plt.title(name)
    plt.ylabel('Average Error')
    plt.xlabel('opportunity')
    plt.show()
    
def apply_threshold(kc,count_thresh):
    counts = np.array(kc['Counts'], dtype=np.float)
    l = len(counts)
    if(count_thresh > 0):
        indx = list(np.nonzero(counts < count_thresh)[0])
        if(len(indx) > 0): l = int(indx[0])
    new_kc = {}
    new_kc['Counts'] = kc['Counts'][:l]
    new_kc['Errors'] = kc['Errors'][:l]
    new_kc['Selection Errors'] = kc['Selection Errors'][:l]
    new_kc['Input Errors'] = kc['Input Errors'][:l]
    new_kc['Hint Errors'] = kc['Hint Errors'][:l]
    return new_kc
    
#marker="+"
def add_counts(a,b):
    if len(a) < len(b):
        c = b.copy()
        c[:len(a)] += a
    else:
        c = a.copy()
        c[:len(b)] += b
    return c
def aggregate_curves(d, selection=None):
    if(selection == None): selection = d.keys();
    print(selection)
    agg_curve = {}
    for (kc_name, kc) in d.items() :
        if(kc_name in selection):
            assert 'Counts' in kc and 'Errors' in kc and 'Selection Errors' in kc and 'Input Errors' in kc and 'Hint Errors' in kc, "bad keys %s:%s" % (kc_name, str(kc.keys()))
            counts = np.array(kc['Counts'], dtype=np.float)
            errors = np.array(kc['Errors'], dtype=np.float)
            s_errors = np.array(kc['Selection Errors'], dtype=np.float)
            i_errors = np.array(kc['Input Errors'], dtype=np.float)
            h_errors = np.array(kc['Hint Errors'], dtype=np.float)
            agg_curve['Counts'] = add_counts(agg_curve.get('Counts',np.zeros(0)), counts)
            agg_curve['Errors'] = add_counts(agg_curve.get('Errors',np.zeros(0)), errors)
            agg_curve['Selection Errors'] = add_counts(agg_curve.get('Selection Errors',np.zeros(0)), s_errors)
            agg_curve['Input Errors'] = add_counts(agg_curve.get('Input Errors',np.zeros(0)), i_errors)
            agg_curve['Hint Errors'] = add_counts(agg_curve.get('Hint Errors',np.zeros(0)), h_errors)
    return agg_curve
            

In [ ]:
graph_kcs(None, {"Control":aggregate_curves(control_curves), "Refactored": aggregate_curves(refactor_curves), 'light_refactor': aggregate_curves(light_refactor_curves), "final": aggregate_curves(final_curves)})
for ((kc_name, control),(_, refactor),(_, light),(_, final)) in zip(control_curves.items(),refactor_curves.items(),light_refactor_curves.items(),final_curves.items()):
    graph_kcs(kc_name,{'control': control, "refactor":refactor, "light":light, "final":final}) 
    

In [ ]:
print(human_curves.keys())
graph_kcs(None, {"No Pretraining":aggregate_curves(control_curves),"Estimated Whole Number":aggregate_curves(substep_curves),"Demonstrated Pretest":aggregate_curves(pretest_curves), "Estimated Fraction":aggregate_curves(iso_curves), "Human":aggregate_curves(human_curves),})


graph_split("All Knowledge Components -- No Pretraining", aggregate_curves(control_curves))
#for (kc_name, kc) in control_curves.items() :
#    graph_split(kc_name, kc)
#print("----------------------------")
#print(aggregate_curves(human_curves))
graph_split("All Knowledge Components -- Estimated Whole Number", aggregate_curves(substep_curves))
graph_split("All Knowledge Components -- Demonstrated Pretest", aggregate_curves(pretest_curves))
graph_split("All Knowledge Components -- Estimated Fraction", aggregate_curves(iso_curves))
#for (kc_name, kc) in iso_curves.items():
#    graph_split(kc_name, kc) 

graph_split("All Knowledge Components -- Human", aggregate_curves(human_curves))


graph_kcs("All Knowledge Components", {"All Knowledge Components -- Estimated Fraction":aggregate_curves(iso_curves), "All Knowledge Components -- Human":aggregate_curves(human_curves),})
for ((kc_name, kc),(kc_name_human, kc_human)) in zip(iso_curves.items(),human_curves.items()):
    graph_kcs(kc_name,{'human': kc_human, "iso":kc}) 
    print(kc_human['Counts'])
    print(kc['Counts'])
    
    
#graph_kcs("All KCs", {"All KCs -- iso":aggregate_curves(iso_curves), "All KCs -- control":aggregate_curves(control_curves),})
#for ((kc_name, kc),(kc_name_human, kc_human)) in zip(iso_curves.items(),control_curves.items()):
#    graph_kcs(kc_name,{'control': kc_human, "iso":kc}) 
#    print(kc_human['Counts'])
#    print(kc['Counts'])

In [ ]:
d = {}
np.median
for ((kc_name, kc),(kc_name_human, kc_human)) in zip(control_curves.items(),human_curves.items()):
    counts = np.array(kc_human['Counts'], dtype=np.float)
    human_error = np.array(kc_human['Errors'], dtype=np.float)/counts
    
    counts = np.array(kc['Counts'], dtype=np.float)
    control_error = np.array(kc['Errors'], dtype=np.float)/counts
    
    inv = list(control_error < human_error[0])
    
    first = list(inv).index(True) if True in inv else len(counts)
    
    l = d.get(kc_name.split()[0],[])
    l.append(first)
    d[kc_name.split()[0]] = l
    
print(d)
print("AS-average:",sum(d['AS'])/float(len(d['AS'])))
print("AD-average:",sum(d['AD'])/float(len(d['AD'])))
print("M-average:",sum(d['M'])/float(len(d['M'])))
print("AS-median:",np.median(d['AS']))
print("AD-median:",np.median(d['AD']))
print("M-median:",np.median(d['M']))
    